## Recommendation Engine For Movies

### Importing Data set

In [6]:
#code to find spark in the system
import findspark
findspark.init('/home/ubuntu/spark-2.4.0-bin-hadoop2.7')

In [7]:
from pyspark.sql import SparkSession

In [8]:
#creating a spark session
spark = SparkSession.builder.appName('movieRec').getOrCreate()

We will be using Collaborative flitering method to make movie recommendations. The below image taken from wikipedia explains how collaborative approach works visually

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [18]:
movie_data = spark.read.csv('ratings.csv',inferSchema=True,header=True)

### Brief view of the data set

In [11]:
movie_data.head()

Row(userId=1, movieId=1, rating=4.0, timestamp=964982703)

In [12]:
movie_data.describe().show()

+-------+------------------+----------------+------------------+--------------------+
|summary|            userId|         movieId|            rating|           timestamp|
+-------+------------------+----------------+------------------+--------------------+
|  count|            100836|          100836|            100836|              100836|
|   mean|326.12756356856676|19435.2957177992| 3.501556983616962|1.2059460873684695E9|
| stddev| 182.6184914635004|35530.9871987003|1.0425292390606342|2.1626103599513078E8|
|    min|                 1|               1|               0.5|           828124615|
|    max|               610|          193609|               5.0|          1537799250|
+-------+------------------+----------------+------------------+--------------------+



There are about 100k users 

Now we will build our model using sparks implementation of collaborative approach using ALS(Alternating least square methods).
We initially divide the data set into training and testing set in a 70-30 fashion randomly. 

In [13]:
(train, test) = movie_data.randomSplit([0.7, 0.3])

In [38]:
# model building
als = ALS(maxIter=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(train)

In [39]:
predictions = model.transform(test)

In [40]:
predictions.show()

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|   133|    471|   4.0| 843491793| 1.6683806|
|   385|    471|   4.0| 850766697| 1.3962233|
|   372|    471|   3.0| 874415126| 3.8238997|
|   218|    471|   4.0|1111624874|  4.889207|
|    57|    471|   3.0| 969753604| 2.0081472|
|   387|    471|   3.0|1139047519| 2.3241572|
|   217|    471|   2.0| 955943727| 1.4605262|
|   176|    471|   5.0| 840109075|  5.853851|
|   136|    471|   4.0| 832450058| 4.7478886|
|   448|    471|   4.0|1178980875| 1.4523933|
|   414|    471|   5.0| 961514069| 3.4358068|
|   426|    471|   5.0|1451081135|  5.455002|
|   541|    471|   3.0| 835643551|  4.225483|
|   260|    471|   4.5|1109409455| 3.1208887|
|   104|    471|   4.5|1238111129| 4.3892865|
|   599|    833|   1.5|1519330029| 1.1072823|
|   609|    833|   3.0| 847221080|0.92107695|
|   492|    833|   4.0| 863976674| 1.2796872|
|   177|   1088|   3.5|1435534616|

### Evaluate the model

In [42]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.1444354482056192


### Making predictions

In [43]:
#Let's say we want to give some recommendations to user 11
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [44]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     36|    11|
|     44|    11|
|    153|    11|
|    170|    11|
|    318|    11|
|    368|    11|
|    480|    11|
|    511|    11|
|    589|    11|
|    736|    11|
|    780|    11|
|   1210|    11|
|   1358|    11|
|   1370|    11|
|   1391|    11|
|   1408|    11|
|   1438|    11|
|   1552|    11|
|   1573|    11|
|   1584|    11|
+-------+------+
only showing top 20 rows



In [47]:
recommended_movies = model.recommendForUserSubset(single_user,5)

In [55]:
recommended_movies.head()

Row(userId=11, recommendations=[Row(movieId=2881, rating=7.605807304382324), Row(movieId=3566, rating=7.557867050170898), Row(movieId=3513, rating=7.507314682006836), Row(movieId=5673, rating=7.3620781898498535), Row(movieId=2186, rating=7.28447151184082)])